# Transfer Learning YOLOv2 on LISA Dataset
---------------------------------------------------------------------


* What is Transfer learning
* Relationship between COCO and LISA

In [1]:
import sys
print(sys.version) # Check Python Version
import os
from keras.optimizers import Adam
from cfg import *

2.7.13 |Anaconda 4.4.0 (64-bit)| (default, Dec 20 2016, 23:09:15) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]


Using TensorFlow backend.


### Prepare LISA Dataset

In [2]:
import numpy as np
from utils.parse_input import load_data    # Data handler for LISA dataset

x_train, y_train = load_data('training.txt')
labels           = np.unique(y_train[:,1])
num_classes      = len(labels)            # Count number of classes in the dataset

print("Train: {} samples\nNumber of classes: {}".format(len(x_train),num_classes))
print("\nLabel Sample: \n{}".format(y_train[0]))

Number of ground truth boxes: 3672 boxes
Train: 3672 samples
Number of classes: 31

Label Sample: 
[1093.5 408.0 45.0 48.0 'doNotEnter']


### Load Pretrained YOLOv2 on COCO Dataset

In [3]:
from model.yolov2 import YOLOv2, darknet19
import keras.backend as K
K.clear_session() # Avoid duplicate model 

# Original YOLOv2 - Trained on COCO Dataset
darknet19 = darknet19(freeze_layers=True)
yolov2    = YOLOv2(feature_extractor=darknet19, num_anchors=5, num_classes=80)
yolov2.model.load_weights('../yolo-coco.h5')
yolov2.model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, None, None, 3) 0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, None, None, 32 864         input_1[0][0]                    
____________________________________________________________________________________________________
batch_normalization_1 (BatchNorm (None, None, None, 32 128         conv2d_1[0][0]                   
____________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)        (None, None, None, 32 0           batch_normalization_1[0][0]      
___________________________________________________________________________________________

## Transfer Learning - Cut last layer and create a new Conv2D for Traffic Sign Detection

In [4]:
from keras.layers import Conv2D
from keras.models import Model
from keras.regularizers import l2

# Cut last layer
topless_model = yolov2.model.layers[-2].output   

# dd a few new Conv2D layers for Traffic Sign Detection
traffic_sign_detector = Conv2D(filters=512,padding='same',
                               kernel_size=(3, 3), 
                               kernel_initializer='he_uniform',
                               activation='relu')(topless_model)

traffic_sign_detector = Conv2D(filters=(N_ANCHORS* (N_CLASSES + 5)),
                               kernel_size=(1, 1), 
                               kernel_regularizer=l2(5e-4), 
                               kernel_initializer='he_uniform', activation='linear',
                               name='yolov2')(traffic_sign_detector)

# Build a new Model for training
model = Model(yolov2.model.input, traffic_sign_detector)

#   # # Freeze lower layers
for layer in yolov2.model.layers[:-16]:
    layer.trainable = False
      
model.summary()
# for l in model.layers:
#     print(l.trainable)

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, None, None, 3) 0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, None, None, 32 864         input_1[0][0]                    
____________________________________________________________________________________________________
batch_normalization_1 (BatchNorm (None, None, None, 32 128         conv2d_1[0][0]                   
____________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)        (None, None, None, 32 0           batch_normalization_1[0][0]      
___________________________________________________________________________________________

In [5]:
import keras.backend as K
import tensorflow as tf

def custom_loss(y_true, y_pred):
    
    SCALE_NOOB, SCALE_CONF, SCALE_COOR, SCALE_PROB = 0.5, 5.0, 5.0, 1.0
    pred_shape = K.shape(y_pred)[1:3]
    gt_shape   = K.shape(y_true)                 # shape of ground truth value
    GRID_H = tf.cast(pred_shape[0], tf.int32)  # shape of output feature map
    GRID_W = tf.cast(pred_shape[1], tf.int32)
    
    output_size = tf.cast(tf.reshape([GRID_W, GRID_H], [1,1,1,1,2]), tf.float32)
    y_pred = tf.reshape(y_pred,[-1, pred_shape[0], pred_shape[1], N_ANCHORS, N_CLASSES +5])
    y_true = tf.reshape(y_true,[-1, gt_shape[1], gt_shape[2], N_ANCHORS, N_CLASSES + 5])

    c_xy = _create_offset_map(K.shape(y_pred))
    
    pred_box_xy = (tf.sigmoid(y_pred[:,:,:,:,:2]) + c_xy)  / output_size
    pred_box_wh = tf.exp(y_pred[:,:,:,:,2:4]) * np.reshape(ANCHORS, [1,1,1,N_ANCHORS,2])
    pred_box_wh = tf.sqrt(pred_box_wh / output_size)
    
    # adjust confidence
    pred_box_conf = tf.expand_dims(tf.sigmoid(y_pred[:, :, :, :, 4]), -1) 
    # adjust probability
    pred_box_prob = tf.nn.softmax(y_pred[:, :, :, :, 5:])
    
    y_pred = tf.concat([pred_box_xy, pred_box_wh, pred_box_conf, pred_box_prob], 4)
    
    ### Adjust ground truth
    center_xy = y_true[:,:,:,:,0:2]
    true_box_xy = center_xy 
    true_box_wh = tf.sqrt(y_true[:,:,:,:,2:4])
    
    # adjust confidence
    pred_tem_wh = tf.pow(pred_box_wh, 2) *  output_size
    pred_box_area = pred_tem_wh[:,:,:,:,0] * pred_tem_wh[:,:,:,:,1]
    pred_box_ul = pred_box_xy - 0.5 * pred_tem_wh
    pred_box_bd = pred_box_xy + 0.5 * pred_tem_wh
    
    true_tem_wh = tf.pow(true_box_wh, 2) *  output_size
    true_box_area = true_tem_wh[:,:,:,:,0] * true_tem_wh[:,:,:,:,1]
    true_box_ul = true_box_xy - 0.5 * true_tem_wh
    true_box_bd = true_box_xy + 0.5 * true_tem_wh
    
    intersect_ul = tf.maximum(pred_box_ul, true_box_ul) 
    intersect_br = tf.minimum(pred_box_bd, true_box_bd)
    intersect_wh = intersect_br - intersect_ul
    intersect_wh = tf.maximum(intersect_wh, 0.0)
    intersect_area = intersect_wh[:,:,:,:,0] * intersect_wh[:,:,:,:,1]
    
    iou = tf.truediv(intersect_area, true_box_area + pred_box_area - intersect_area)
    best_box = tf.equal(iou, tf.reduce_max(iou, [3], True)) 
    best_box = tf.to_float(best_box)
    true_box_conf = tf.expand_dims(best_box * y_true[:,:,:,:,4], -1)
    
    # adjust confidence
    true_box_prob = y_true[:,:,:,:,5:]
    
    y_true = tf.concat([true_box_xy, true_box_wh, true_box_conf, true_box_prob], 4)
    
    ### Compute the weights
    weight_coor = tf.concat(4 * [true_box_conf], 4)
    weight_coor = SCALE_COOR * weight_coor
    weight_conf = SCALE_NOOB * (1. - true_box_conf) + SCALE_CONF * true_box_conf
    weight_prob = tf.concat(N_CLASSES * [true_box_conf], 4) 
    weight_prob = SCALE_PROB * weight_prob 
    
    weight = tf.concat([weight_coor, weight_conf, weight_prob], 4)
    
    ### Finalize the loss
    loss = tf.pow(y_pred - y_true, 2)
    loss = loss * weight                     
    total_loss = tf.reshape(loss, [-1, tf.cast(GRID_W*GRID_H, tf.int32)*N_ANCHORS*(4 + 1 + N_CLASSES)])
    total_loss = tf.reduce_sum(total_loss, 1)
    total_loss = .5 * tf.reduce_mean(total_loss)
    
    # @TODO update scalar loss
    localization_loss =  tf.reduce_mean(tf.reduce_sum(loss[..., 0]))
    confidence_loss   =  tf.reduce_mean(tf.reduce_sum(loss[..., 1]))
    classification_loss =tf.reduce_mean(tf.reduce_sum(loss[..., 2]))
                                        
    tf.summary.scalar('average_iou', tf.reduce_mean(tf.reduce_sum(iou)))
    tf.summary.scalar('localization_loss', localization_loss)
    tf.summary.scalar('confidence_loss', confidence_loss)
    tf.summary.scalar('classification_loss', classification_loss)
    
    return total_loss

def _create_offset_map(output_shape):
    """
    In Yolo9000 paper, grid map
    """
    GRID_H = tf.cast(output_shape[1], tf.int32)  # shape of output feature map
    GRID_W = tf.cast(output_shape[2], tf.int32)
    
    cx = tf.cast((K.arange(0, stop=GRID_W)), dtype=tf.float32)
    cx = K.tile(cx, [GRID_H])
    cx = K.reshape(cx, [-1, GRID_H, GRID_W, 1])

    cy = K.cast((K.arange(0, stop=GRID_H)), dtype=tf.float32)
    cy = K.reshape(cy, [-1, 1])
    cy = K.tile(cy, [1, GRID_W])  
    cy = K.reshape(cy, [-1])    
    cy = K.reshape(cy, [-1, GRID_H, GRID_W, 1])

    c_xy = tf.stack([cx, cy], -1)
    c_xy = K.cast(c_xy, tf.float32)

    return c_xy

def avg_iou(y_true, y_pred):
    pred_shape = K.shape(y_pred)[1:3]
    gt_shape   = K.shape(y_true)                 # shape of ground truth value
    GRID_H = tf.cast(pred_shape[0], tf.int32)  # shape of output feature map
    GRID_W = tf.cast(pred_shape[1], tf.int32)
    
    output_size = tf.cast(tf.reshape([GRID_W, GRID_H], [1,1,1,1,2]), tf.float32)
    y_pred = tf.reshape(y_pred,[-1, pred_shape[0], pred_shape[1], N_ANCHORS, N_CLASSES +5])
    y_true = tf.reshape(y_true,[-1, gt_shape[1], gt_shape[2], N_ANCHORS, N_CLASSES + 5])

    c_xy = _create_offset_map(K.shape(y_pred))
    
    pred_box_xy = (tf.sigmoid(y_pred[:,:,:,:,:2]) + c_xy)  / output_size
    pred_box_wh = tf.exp(y_pred[:,:,:,:,2:4]) * np.reshape(ANCHORS, [1,1,1,N_ANCHORS,2])
    pred_box_wh = tf.sqrt(pred_box_wh / output_size)
    
    # adjust confidence
    pred_box_conf = tf.expand_dims(tf.sigmoid(y_pred[:, :, :, :, 4]), -1) 
    # adjust probability
    pred_box_prob = tf.nn.softmax(y_pred[:, :, :, :, 5:])
    
    y_pred = tf.concat([pred_box_xy, pred_box_wh, pred_box_conf, pred_box_prob], 4)
    
    ### Adjust ground truth
    center_xy = y_true[:,:,:,:,0:2]
    true_box_xy = center_xy 
    true_box_wh = tf.sqrt(y_true[:,:,:,:,2:4])
    
    # adjust confidence
    pred_tem_wh = tf.pow(pred_box_wh, 2) *  output_size
    pred_box_area = pred_tem_wh[:,:,:,:,0] * pred_tem_wh[:,:,:,:,1]
    pred_box_ul = pred_box_xy - 0.5 * pred_tem_wh
    pred_box_bd = pred_box_xy + 0.5 * pred_tem_wh
    
    true_tem_wh = tf.pow(true_box_wh, 2) *  output_size
    true_box_area = true_tem_wh[:,:,:,:,0] * true_tem_wh[:,:,:,:,1]
    true_box_ul = true_box_xy - 0.5 * true_tem_wh
    true_box_bd = true_box_xy + 0.5 * true_tem_wh
    
    intersect_ul = tf.maximum(pred_box_ul, true_box_ul) 
    intersect_br = tf.minimum(pred_box_bd, true_box_bd)
    intersect_wh = intersect_br - intersect_ul
    intersect_wh = tf.maximum(intersect_wh, 0.0)
    intersect_area = intersect_wh[:,:,:,:,0] * intersect_wh[:,:,:,:,1]
    
    iou = tf.truediv(intersect_area, true_box_area + pred_box_area - intersect_area)
    
    return tf.reduce_sum(iou) / tf.to_float(gt_shape[0])

In [6]:
# over fitting on 1 iamge
from sklearn.utils import shuffle
x_train, y_train = shuffle(x_train, y_train)

In [7]:
one_sample = np.tile(x_train[0:10], 2).tolist()
one_label  = np.tile(y_train[0:10], [2, 1])
print([name.split('/')[-1].split('.')[0] for name in one_sample])
print(one_label)
print(one_sample[5])

['signalAhead_1405107045', 'speedLimit50_1405372499', 'keepRight_1405359763', 'stop_1398993494', 'speedLimit30_1405371806', 'speedLimit25_1404948650', 'curveRight_1398812169', 'stop_1404942178', 'noUTurn_1405359264', 'signalAhead_1405111415', 'signalAhead_1405107045', 'speedLimit50_1405372499', 'keepRight_1405359763', 'stop_1398993494', 'speedLimit30_1405371806', 'speedLimit25_1404948650', 'curveRight_1398812169', 'stop_1404942178', 'noUTurn_1405359264', 'signalAhead_1405111415']
[[964.5 463.0 47.0 50.0 'signalAhead']
 [370.0 450.0 32.0 42.0 'speedLimit50']
 [442.5 520.0 17.0 24.0 'keepRight']
 [1105.0 285.5 180.0 169.0 'stop']
 [958.5 435.0 25.0 38.0 'speedLimit30']
 [933.0 465.0 34.0 42.0 'speedLimit25']
 [802.5 466.5 27.0 25.0 'speedLimit35']
 [948.5 461.5 47.0 53.0 'stop']
 [1150.5 416.5 37.0 39.0 'noUTurn']
 [849.0 428.5 60.0 61.0 'signalAhead']
 [964.5 463.0 47.0 50.0 'signalAhead']
 [370.0 450.0 32.0 42.0 'speedLimit50']
 [442.5 520.0 17.0 24.0 'keepRight']
 [1105.0 285.5 180.0 

In [ ]:
import os
import keras
from utils.multi_gpu import make_parallel, get_gpus
from utils.data_generator import flow_from_list
# from sklearn.model_selection import train_test_split

# # HYPER-PARAMETERS
BATCH_SIZE = 8
EPOCHS     = 1
LEARN_RATE = 0.0001 # this model has been pre-trained, LOWER LR is needed

# Data Generator
train_data_gen = flow_from_list(one_sample,one_label, batch_size=BATCH_SIZE, augment_data=True)
val_data_gen = flow_from_list(one_sample,one_label, batch_size=BATCH_SIZE, augment_data=False)

In [ ]:
from keras.optimizers import Adam, SGD
import keras

backup_path = '/home/ubuntu/dataset/backup/'
# For Debugging purpose
tf_board   = keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=0, write_graph=True, write_images=False)
early_stop = keras.callbacks.EarlyStopping(monitor='loss', min_delta=0.0000001, patience=3, mode='min', verbose=1)
save_model = keras.callbacks.LambdaCallback(on_epoch_end=lambda epoch,logs: model.save_weights(backup_path + 'yolov2-epoch%s-loss%s.weights'%(epoch, str(logs.get('loss')))))


# Load pretrain weights
# model.load_weights("yolov2.weights")

# Set up Data parallelism
n_gpus = get_gpus()
if n_gpus > 1:
    BATCH_SIZE = n_gpus * BATCH_SIZE
    model_par = make_parallel(model, n_gpus)
else:
    model_par = model

# Set up optimizer


adam = Adam(LEARN_RATE)
# sgd = SGD(LEARN_RATE, decay=0.0005, momentum=0.9)
def schedule(epochs):
    if epochs < 20:
        return LEARN_RATE
    if 20 <= epochs <= 25:
        return LEARN_RATE / 10.
    if epochs >= 25:
        return LEARN_RATE / 100.


lr_scheduler = keras.callbacks.LearningRateScheduler(schedule)
   
# Compiling model
model_par.compile(optimizer=adam,loss=custom_loss, metrics=[avg_iou])

hist =  model_par.fit_generator(generator     = train_data_gen, 
                            steps_per_epoch   = 5, 
                            validation_data   = val_data_gen,
                            validation_steps  = 2,
                            epochs            = 30, 
                            callbacks         = [tf_board, lr_scheduler],
                            workers=1, verbose=1)

model.save_weights('yolov2.weights')

There is(are) 1 GPU(s) on device.
Epoch 1/30
5/5 [==============================] - 10s - loss: 40.3352 - avg_iou: 0.8715 - val_loss: 3.2683 - val_avg_iou: 0.9353
Epoch 2/30
5/5 [==============================] - 6s - loss: 2.9651 - avg_iou: 1.0881 - val_loss: 2.5635 - val_avg_iou: 1.1127
Epoch 3/30
5/5 [==============================] - 6s - loss: 2.2156 - avg_iou: 1.1629 - val_loss: 2.8284 - val_avg_iou: 1.0292
Epoch 4/30
5/5 [==============================] - 6s - loss: 2.4552 - avg_iou: 0.9283 - val_loss: 2.6072 - val_avg_iou: 1.1954
Epoch 5/30
5/5 [==============================] - 6s - loss: 1.5405 - avg_iou: 1.2168 - val_loss: 2.1893 - val_avg_iou: 1.2450
Epoch 6/30
5/5 [==============================] - 6s - loss: 2.1339 - avg_iou: 1.0827 - val_loss: 1.8602 - val_avg_iou: 1.1817
Epoch 7/30
5/5 [==============================] - 6s - loss: 1.6794 - avg_iou: 1.0985 - val_loss: 1.2749 - val_avg_iou: 1.2000
Epoch 8/30
5/5 [==============================] - 6s - loss: 1.5372 - avg_i

In [ ]:
model.save('model.h5')

## Multi-GPUs Training - Data Parallelism Approach

* Each GPU will have a copy of the model
* During training time, mean of all gradidents from each GPU will be calculated to update the model
<img style="width:40%" src="https://www.tensorflow.org/images/Parallelism.png">

In [ ]:
model.load_weights('yolov2.weights')

### Visualize training process using Tensorboard
Open `http://<public-dns>:6006`

In [ ]:
# import matplotlib.pyplot as plt
# %matplotlib inline
# samples, labels = train_data_gen.next()
# i = 2
# img    = samples[i]
# labels = labels.reshape([-1, 30, 40, 5, 36])
# box    = labels[i][8][30][0][:4]*np.array([1280, 960, 1280, 960])
# print(box)

# dot    = cv2.circle(img, (int(box[0]), int(box[1])), radius=5, color=[0, 1, 0], thickness=5)

# plt.figure(figsize=(10, 10))
# plt.imshow(dot)

In [ ]:
# import cv2
# import pandas as pd
# import numpy as np
# from sklearn.utils import shuffle
# import matplotlib.pyplot as plt
# %matplotlib inline

# def extract_sign(img, bbox, output_size=(32, 32)):
#     xc, yc, w, h = bbox.x, bbox.y, bbox.w, bbox.h
#     x1   = int(xc - w/2)
#     y1   = int(yc - h/2)
#     x2   = int(xc + w/2)
#     y2   = int(yc + h/2)
#     roi = cv2.resize(img[y1:y2, x1:x2], output_size)
#     return roi

# x_train, y_train = shuffle(x_train, y_train)
# fig = plt.figure(figsize=(17, 8))
# for i, label in enumerate(labels):
#     ax           = fig.add_subplot(4, 8, 1 + i, xticks=[], yticks=[])
#     idx          = np.where(y_train[:, 1] == label)[0][0]
#     img          = cv2.cvtColor(cv2.imread(x_train[idx]), cv2.COLOR_BGR2RGB)
#     box          = y_train[idx][0]
#     sign_only    = extract_sign(img, box, (32, 32)) #  just extract the sign
#     ax.set_title(label)
#     plt.imshow(sign_only)
# plt.show()